In [1]:
# recursive character text splitting 
# line change \n 
# \n\n para

text ="""Artificial Intelligence (AI) has rapidly evolved from a futuristic concept to a transformative force deeply integrated into modern life. Initially conceived in the mid-20th century as a theoretical field, AI was aimed at enabling machines to mimic human intelligence. Early AI systems were rule-based and limited in capability, often confined to narrow tasks such as playing chess or solving equations. However, with advances in computing power, access to large data sets, and sophisticated algorithms, AI has experienced exponential growth and widespread adoption.

Today, AI influences nearly every sector of society. In healthcare, AI-powered diagnostics are revolutionizing patient care. Algorithms can analyze medical images, predict disease outbreaks, and even suggest personalized treatments with greater accuracy and speed than human practitioners. AI chatbots are assisting patients in scheduling appointments and answering health-related queries, reducing the burden on healthcare staff.

In the field of education, AI tools personalize learning experiences by adapting content to a student’s pace and understanding. Intelligent tutoring systems provide real-time feedback and suggest improvements, making education more interactive and effective. Moreover, AI-driven language models help bridge communication gaps by providing instant translations and grammar corrections.

The financial sector has also embraced AI to detect fraud, automate trading, and assess creditworthiness. Machine learning models analyze massive volumes of transaction data to flag anomalies and prevent fraudulent activities. Robo-advisors, powered by AI, provide investment guidance to individuals based on their financial goals and risk tolerance.

Transportation is another domain where AI has made significant strides. Self-driving vehicles, though still under development, rely heavily on AI to interpret sensory data, make real-time decisions, and navigate safely. Ride-sharing platforms use AI to optimize routes, reduce wait times, and ensure efficient matching of drivers with passengers. In aviation and logistics, AI systems enhance safety, efficiency, and predictive maintenance.

Despite its benefits, AI also raises several ethical, social, and economic concerns. One of the most pressing issues is the potential for job displacement due to automation. While AI creates new opportunities in data science and machine learning, it also threatens routine, manual, and even some cognitive jobs. This shift necessitates re-skilling and educational reform to prepare the workforce for AI-driven industries.

Bias in AI algorithms is another concern. Since AI systems learn from historical data, they may inherit and even amplify existing societal biases. Ensuring fairness, transparency, and accountability in AI decision-making is crucial to prevent discrimination in areas such as hiring, lending, and law enforcement.

Moreover, the rise of AI has sparked debates about data privacy and surveillance. Intelligent systems collect and analyze personal data to deliver personalized experiences, but without proper regulation, this can lead to misuse and erosion of individual privacy.

In conclusion, AI is undeniably reshaping the way we live, work, and interact. Its impact is far-reaching and multifaceted—enhancing productivity, unlocking innovation, and offering new solutions to age-old problems. However, the path forward requires a balanced approach: embracing the benefits while addressing the challenges through thoughtful policies, inclusive development, and ethical use. The future of AI holds immense promise, and with responsible stewardship, it can be a force for positive and sustainable change in society."""

Resurcive Text Splitter

--Good to brak on the basics of sentence good for context

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# CHUNCK SIZE Allowed is the size is 10 so 
# it break into para then fulstops then to should be low than the 
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100 ,
    chunk_overlap = 10
)
chunk = splitter.split_text(text)
chunk[0]

'Artificial Intelligence (AI) has rapidly evolved from a futuristic concept to a transformative force'

Code spliting using recusive text spliter

In [18]:
code = """import datetime

class Book:
    def __init__(self, title, author, isbn, publication_year):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.publication_year = publication_year

    def display_info(self):
        print(f"Title: {self.title}")
        print(f"Author: {self.author}")
        print(f"ISBN: {self.isbn}")
        print(f"Publication Year: {self.publication_year}")

    def is_recent_publication(self, years_threshold=5):
        current_year = datetime.datetime.now().year
        return (current_year - self.publication_year) <= years_threshold

    def __str__(self):
        return f'"{self.title}" by {self.author} ({self.publication_year})'

if __name__ == "__main__":
    book1 = Book("The Great Gatsby", "F. Scott Fitzgerald", "978-0743273565", 1925)
    book2 = Book("Python Crash Course", "Eric Matthes", "978-1593279288", 2019)
    book3 = Book("Dune", "Frank Herbert", "978-0441172719", 1965)
    book4 = Book("Clean Code", "Robert C. Martin", "978-0132350884", 2008)

    print("--- Book 1 Information ---")
    book1.display_info()
    print(f"Is recent publication? {book1.is_recent_publication()}")
    print(f"String representation: {book1}\n")

    print("--- Book 2 Information ---")
    book2.display_info()
    print(f"Is recent publication (last 5 years)? {book2.is_recent_publication()}")
    print(f"Is recent publication (last 10 years)? {book2.is_recent_publication(10)}\n")

    print("--- Book 3 Information ---")
    print(book3)
    print(f"Is recent publication? {book3.is_recent_publication()}\n")

    print("--- Book 4 Information ---")
    print(book4)
    print(f"Is recent publication? {book4.is_recent_publication()}")
"""

from langchain_text_splitters import Language, RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, 
    chunk_size = 100,
    chunk_overlap = 0
)

result = splitter.split_text(code)
result[1]

'class Book:\n    def __init__(self, title, author, isbn, publication_year):'

Sementic Based distribution

In [20]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Sample multi-domain paragraph
text = """
The farmer used AI to monitor crop yields, while also coaching his village football team using data analytics.
His cows grazed peacefully as he reviewed drone footage.
The AI assistant alerted him about soil dryness and also showed replays of yesterday’s game.
"""

# Step 2: Split into sentences or semantic phrases
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
sentences = sent_tokenize(text)

print("\n🧾 Split Sentences:")
for i, s in enumerate(sentences):
    print(f"{i+1}. {s}")
    print(i)
    print('this is s :',s)

# Step 3: Embed sentences using sentence transformer
model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_embeddings = model.encode(sentences)

# Step 4: Compare sentence similarities for semantic chunking
threshold = 0.65
chunks = []
current_chunk = [sentences[0]]

for i in range(1, len(sentences)):
    sim = cosine_similarity(
        [sentence_embeddings[i]], 
        [model.encode([" ".join(current_chunk)])]
    )[0][0]
    
    if sim >= threshold:
        current_chunk.append(sentences[i])
    else:
        chunks.append(" ".join(current_chunk))
        current_chunk = [sentences[i]]

# Add the last chunk
if current_chunk:
    chunks.append(" ".join(current_chunk))

print("\n🧩 Semantic Chunks:")
for i, chunk in enumerate(chunks):
    print(f"\nChunk {i+1}:\n{chunk}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MUHAMMAD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



🧾 Split Sentences:
1. 
The farmer used AI to monitor crop yields, while also coaching his village football team using data analytics.
0
this is s : 
The farmer used AI to monitor crop yields, while also coaching his village football team using data analytics.
2. His cows grazed peacefully as he reviewed drone footage.
1
this is s : His cows grazed peacefully as he reviewed drone footage.
3. The AI assistant alerted him about soil dryness and also showed replays of yesterday’s game.
2
this is s : The AI assistant alerted him about soil dryness and also showed replays of yesterday’s game.


ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.